In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping


In [71]:
# Load and preprocess data
test_data = pd.read_csv(r'D:\1A Shiash\23-09 to 28 -09\Test Classification Using BERT\DataSet\Corona_NLP_test.csv', encoding='latin-1')
train_data = pd.read_csv(r'D:\1A Shiash\23-09 to 28 -09\Test Classification Using BERT\DataSet\Corona_NLP_train.csv', encoding='latin-1')



In [72]:
# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['Sentiment'])
test_labels = label_encoder.transform(test_data['Sentiment'])



In [73]:
# Convert to categorical
train_labels_cat = tf.keras.utils.to_categorical(train_labels)
test_labels_cat = tf.keras.utils.to_categorical(test_labels)



In [79]:
import os
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy

# Optional: Disable GPU and run on CPU to avoid OOM errors
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Uncomment to disable GPU

# Enable mixed precision training for memory optimization
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'  # Smaller model to prevent OOM
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_classes = 5  # Number of sentiment classes (adjust according to your dataset)

# Load the model with sequence classification
bert_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Define optimizer and loss function
optimizer = Adam(learning_rate=2e-5)
loss_fn = CategoricalCrossentropy(from_logits=True)
train_acc_metric = CategoricalAccuracy()
val_acc_metric = CategoricalAccuracy()

# Compile the model
bert_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[train_acc_metric])


ResourceExhaustedError: OOM when allocating tensor with shape[30522,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

In [ ]:
# Build a custom model
def build_model(num_classes):
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
    pooled_output = bert_outputs[0][:, 0, :]  # CLS token output

    x = Dropout(0.3)(pooled_output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model


In [ ]:
# Tokenize and encode the text data
max_length = 64

def encode_text(texts):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf'
    )

train_encodings = encode_text(train_data['OriginalTweet'].tolist())
test_encodings = encode_text(test_data['OriginalTweet'].tolist())

# Freeze BERT layers (we will only train the classifier)
bert_model.trainable = False

In [52]:
# Build the model
def build_model(num_classes):
    input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    roberta = TFAutoModel.from_pretrained(model_name)
    outputs = roberta(input_ids, attention_mask=attention_mask)
    pooled_output = outputs[0][:, 0, :]  # Use the [CLS] token representation

    x = Dropout(0.1)(pooled_output)
    x = Dense(128, activation='relu')(x)  # Reduced hidden layer size
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)  # Ensure final layer uses float32

    model = Model(inputs=[input_ids, attention_mask], outputs=outputs)
    return model

In [54]:
# Create and compile the model
num_classes = len(label_encoder.classes_)
model = build_model(num_classes)

# Set the optimizer, loss, and metrics
optimizer = Adam(learning_rate=2e-5)
loss_fn = CategoricalCrossentropy(from_logits=False)
train_acc_metric = CategoricalAccuracy()
val_acc_metric = CategoricalAccuracy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=[train_acc_metric])

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [ ]:
# Prepare the data
batch_size = 4  # Further reduce batch size to 4 to save memory
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    train_labels_cat
)).shuffle(1000).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    test_labels_cat
)).batch(batch_size)


In [15]:
# Training loop
epochs = 10
steps_per_epoch = 500  # Set a fixed number of steps per epoch
target_accuracy = 0.80

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # Training
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset.take(steps_per_epoch)):
        loss_value, accuracy = model.train_on_batch(x_batch_train, y_batch_train)

        if step % 100 == 0:
            print(f"Step {step}/{steps_per_epoch}: Loss: {loss_value:.4f}, Accuracy: {accuracy:.4f}")

    # Validation
    val_losses = []
    val_accuracies = []
    for x_batch_val, y_batch_val in val_dataset:
        val_loss, val_accuracy = model.test_on_batch(x_batch_val, y_batch_val)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

    val_loss = np.mean(val_losses)
    val_accuracy = np.mean(val_accuracies)
    print(f"Training Accuracy: {accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

    if accuracy >= target_accuracy:
        print(f"Reached target accuracy of {target_accuracy:.2%}. Stopping training.")
        break




Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model/tf_roberta_model/roberta/encoder/layer_._11/attention/self/dropout_33/dropout/random_uniform/RandomUniform' defined at (most recent call last):
    File "c:\Users\srini\anaconda3\envs\Ten\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\srini\anaconda3\envs\Ten\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\srini\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\srini\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "C:\Users\srini\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "C:\Users\srini\AppData\Roaming\Python\Python310\site-packages\tornado\platfo

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    test_labels_cat
)
print(f"Test accuracy: {test_accuracy:.4f}")



In [ ]:
# Function to predict sentiment for new tweets
def predict_sentiment(tweets):
    encodings = encode_text(tweets)
    predictions = model.predict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask']
    })
    return label_encoder.inverse_transform(np.argmax(predictions, axis=1))



In [ ]:
# Example usage
new_tweets = [
    "I'm worried about the increasing number of COVID-19 cases",
    "The vaccine rollout gives me hope for the future.",
    "Wearing masks is an effective way to prevent the spread of the virus."
]

predicted_sentiments = predict_sentiment(new_tweets)
for tweet, sentiment in zip(new_tweets, predicted_sentiments):
    print(f"Tweet: {tweet}")
    print(f"Predicted sentiment: {sentiment}\n")

# Print the label encoding
print("Label Encoding:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{i}: {label}")